Author: Omar El Malki (omar.elmalki@epfl.ch)

### Coreference resolution on ROCStories

In [1]:
import pandas as pd
import spacy
import neuralcoref

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [2]:
# Read ROCStories into pandas DataFrame
roc_stories_path_csv = "../data/rocstories-2017/ROCStories_winter2017.csv"
roc_stories_df = pd.read_csv(roc_stories_path_csv, sep=',', header=0)

nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)

In [3]:
# Example story
s1 = "David noticed he had put on a lot of weight recently."
s2 = "He examined his habits to try and figure out the reason."
s3 = "He realized he'd been eating too much fast food lately."
s4 = "He stopped going to burger places and started a vegetarian diet."
s5 = "After a few weeks, he started to feel much better."

In [4]:
def resolve_story(*args: str):
    """
    Return coreference clusters and resolved list of sentences from a list oof input sentences

    :param args: input sentences
    :return: coreference clusters
    :return: resolved sentences
    """
    story = ""
    n = len(args)

    for i in range(n):
        story += args[i] 
        if i != n-1:
            story += ". "

    coref_res = nlp(story)
    result = [x for x in coref_res._.coref_resolved.split(". ")]
    return coref_res._.coref_clusters, result

In [5]:
resolve_story(s1, s2, s3, s4, s5)

([David: [David, he, He, his, He, he, He, he]],
 ['David noticed David had put on a lot of weight recently.',
  'David examined David habits to try and figure out the reason.',
  "David realized David'd been eating too much fast food lately.",
  'David stopped going to burger places and started a vegetarian diet.',
  'After a few weeks, David started to feel much better.'])

In [6]:
coref_df = roc_stories_df.head(10)

In [7]:
# Apply resolution to all rows
coref_df['coref_result'] = coref_df.apply(lambda row: resolve_story(row.sentence1, row.sentence2, row.sentence3, row.sentence4, row.sentence5), axis=1)

# Add coreference clusters to dataframe
coref_df['coref_clusters'] = coref_df['coref_result'].apply(lambda x: x[0])

# Add Resolved sentence to Dataframe for each sentence in the dataset
for i in range(1,6):
    coref_df[f'resolved{i}'] = coref_df['coref_result'].apply(lambda x: x[1][i-1])
del coref_df['coref_result']

In [8]:
coref_df

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5,coref_clusters,resolved1,resolved2,resolved3,resolved4,resolved5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight recently.,He examined his habits to try and figure out the reason.,He realized he'd been eating too much fast food lately.,He stopped going to burger places and started a vegetarian diet.,"After a few weeks, he started to feel much better.","[((David), (he), (He), (his), (He), (he), (He), (he))]",David noticed David had put on a lot of weight recently.,David examined David habits to try and figure out the reason.,David realized David'd been eating too much fast food lately.,David stopped going to burger places and started a vegetarian diet.,"After a few weeks, David started to feel much better."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about his actions.,"[((Tom), (him), (He), (his), (Tom), (Tom), (his), (his))]",Tom had a very short temper.,One day a guest made Tom very angry.,Tom punched a hole in the wall of Tom house.,Tom's guest became afraid and left quickly.,Tom sat on Tom couch filled with regret about Tom actions.
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual event.,All of his clothes were either too formal or too casual.,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for the event.,"[((Marcus), (his), (He), (he), (him), (Marcus)), ((a, pair, of, khakis), (The, pair, he, bought))]",Marcus needed clothing for a business casual event.,All of Marcus clothes were either too formal or too casual.,Marcus decided to buy a pair of khakis.,a pair of khakis fit Marcus perfectly.,Marcus was happy to have the right clothes for the event.
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and haul it with his car.,Bill thought a truck would be better for what he needed.,Bobby pointed out two vehicles were much more expensive.,Bill was set in his ways with conventional thinking.,He ended up buying the truck he wanted despite Bobby's advice.,"[((Bobby), (Bobby), (Bobby)), ((Bill), (his), (Bill), (he), (Bill), (his), (He), (he)), ((a, trailer), (it)), ((a, truck), (the, truck, he, wanted))]",Bobby thought Bill should buy a trailer and haul a trailer with Bill car.,Bill thought a truck would be better for what Bill needed.,Bobby pointed out two vehicles were much more expensive.,Bill was set in Bill ways with conventional thinking.,Bill ended up buying a truck despite Bobby's advice.
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in advance but to no avail.,He decided to learn to sing to overcome his handicap.,He then made all his sermons into music and sang them on Sundays.,His congregation was delighted and so was he.,"[((John), (He), (his), (He), (his), (He), (his), (His), (he)), ((all, his, sermons), (them))]",John was a pastor with a very bad memory.,John tried to memorize John sermons many days in advance but to no avail.,John decided to learn to sing to overcome John handicap.,John then made all John sermons into music and sang all his sermons on Sundays.,John congregation was delighted and so was John.
5,4d7b022e-25d2-4300-a9b0-24ab35f4045b,Melody's trip to the aquarium.,Melody's parents surprised her with a trip to the big aquarium.,Melody took a nap during the two hour car ride to the aquarium.,"When they arrived, Melody was energetic and excited.","At the aquarium Melody saw sharks, tropical fish and many others.","After five hours at the aquarium, Melody and her family drove home.","[((Melody), (her)), ((Melody, 's, 